In [ ]:
#2D Bin Packing with Rectpack and a Genetic Algorithm
#Part 1: Baseline using rectpack only
#Part 2: Genetic algorithm that searches over item order and rotations, using rectpack as the packing engine.

#This code is general, you can adapt it with your values ​​and your CSV file.

#Common imports and helpers

import math
import random
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from rectpack import newPacker, SORT_NONE



# Part 1 — Baseline: 2D bin packing using rectpack


# Configuration
CSV_PATH = "data/example_items.csv"  # input CSV (you can change this)
SHOW_PLOTS = True                   # set to False if you don't want plots
NB_BINS_MAX = 1000                  # upper bound, rectpack will use only what is needed

# Expected CSV columns (generic names)
COL_BATCH = "batch_id"
COL_ITEM_W = "item_width"
COL_ITEM_H = "item_height"
COL_PANEL_W = "panel_width"
COL_PANEL_H = "panel_height"
COL_IS_MANDATORY = "is_mandatory"   # 1 if mandatory, 0 otherwise (optional column)


def load_items_by_batch(csv_path: str) -> Dict[str, pd.DataFrame]:
    """Load items from a CSV and group them by batch_id."""
    df = pd.read_csv(csv_path)
    # If no batch column exists, fake a single batch
    if COL_BATCH not in df.columns:
        df[COL_BATCH] = "batch_0"
    batches = {k: g.reset_index(drop=True) for k, g in df.groupby(COL_BATCH, sort=False)}
    return batches


def pack_batch_rectpack(df_batch: pd.DataFrame):
    """
    Pack a single batch of items using rectpack (baseline).
    Returns:
        bins: list of (bin_index, bin_width, bin_height, [(x, y, w, h, item_idx), ...])
    """
    # Build list of items
    items = list(
        zip(
            df_batch[COL_ITEM_W].astype(float).tolist(),
            df_batch[COL_ITEM_H].astype(float).tolist(),
            range(len(df_batch)),
        )
    )

    # Assume all items of a batch use the same panel type(s):
    # here we read from the rows, you can adapt to your real data.
    panel_types = df_batch[[COL_PANEL_W, COL_PANEL_H]].drop_duplicates().values.tolist()

    packer = newPacker(rotation=True)  # baseline: allow rotation, default sort

    # Add items (w, h, id)
    for w, h, idx in items:
        packer.add_rect(w, h, idx)

    # Add bins (panel types)
    for w_panel, h_panel in panel_types:
        packer.add_bin(w_panel, h_panel, count=NB_BINS_MAX)

    # Run packing
    packer.pack()

    # Collect results
    bins = []
    for bin_index, b in enumerate(packer):
        bw, bh = b.width, b.height
        rects_info = []
        for rect in b:
            x, y = rect.x, rect.y
            w, h = rect.width, rect.height
            item_id = rect.rid
            rects_info.append((x, y, w, h, item_id))
        bins.append((bin_index, bw, bh, rects_info))

    return bins


def compute_waste_metrics(df_batch: pd.DataFrame, bins) -> Dict[str, float]:
    """Compute basic utilization / waste statistics for a batch."""
    # Panel area used
    total_panel_area = 0.0
    for _, bw, bh, _ in bins:
        total_panel_area += bw * bh

    # Real used area = sum of item areas
    real_used_area = (
        df_batch[COL_ITEM_W].astype(float) * df_batch[COL_ITEM_H].astype(float)
    ).sum()

    utilization = real_used_area / total_panel_area if total_panel_area > 0 else 0.0
    waste = 1.0 - utilization
    return {
        "total_panel_area": total_panel_area,
        "real_used_area": real_used_area,
        "utilization": utilization,
        "waste": waste,
    }


def plot_packing(bins, df_batch: pd.DataFrame, title: str = "Packing layout"):
    """Plot all bins with items for visualization."""
    if not SHOW_PLOTS:
        return

    for bin_index, bw, bh, rects in bins:
        fig, ax = plt.subplots()
        ax.set_title(f"{title} — Bin {bin_index}")
        ax.set_xlim(0, bw)
        ax.set_ylim(0, bh)
        ax.set_aspect("equal")

        for (x, y, w, h, item_id) in rects:
            rect = patches.Rectangle(
                (x, y),
                w,
                h,
                fill=False,
                linewidth=1,
            )
            ax.add_patch(rect)
            ax.text(
                x + w / 2,
                y + h / 2,
                str(item_id),
                ha="center",
                va="center",
                fontsize=6,
            )

        plt.gca().invert_yaxis()
        plt.show()


def run_baseline():
    """Run the baseline rectpack pipeline on all batches."""
    batches = load_items_by_batch(CSV_PATH)
    for batch_id, df_batch in batches.items():
        print(f"\n=== Baseline: batch {batch_id} ===")
        bins = pack_batch_rectpack(df_batch)
        metrics = compute_waste_metrics(df_batch, bins)
        print(
            f"Utilization: {metrics['utilization']:.3%} — Waste: {metrics['waste']:.3%} "
            f"(panel area: {metrics['total_panel_area']:.1f}, "
            f"item area: {metrics['real_used_area']:.1f})"
        )
        plot_packing(bins, df_batch, title=f"Baseline packing — {batch_id}")






# Part 2 — Genetic algorithm on top of rectpack


# GA hyperparameters
GA_POP_SIZE = 40
GA_N_GENERATIONS = 80
GA_MUTATION_RATE = 0.15
GA_ELITE_FRACTION = 0.1
GA_TOURNAMENT_SIZE = 4


def build_panel_types_from_batch(df_batch: pd.DataFrame) -> List[Tuple[int, float, float]]:
    """
    Build a list of panel types from a batch.
    Here we use a simple single-type assumption:
        [(panel_type_id, width, height)]
    """
    panel_types = df_batch[[COL_PANEL_W, COL_PANEL_H]].drop_duplicates().values.tolist()
    result = []
    for i, (w, h) in enumerate(panel_types):
        result.append((i, float(w), float(h)))
    return result


def pack_given_order(
    items: List[Tuple[float, float]],
    panel_types: List[Tuple[int, float, float]],
    order: List[int],
    rot_flags: List[int],
):
    """
    Pack items using rectpack with a fixed order and given rotation flags.
    Returns:
        bins: list of (bin_index, bw, bh, [(x, y, w, h, item_idx), ...])
    """
    packer = newPacker(rotation=False, sort_algo=SORT_NONE)  # we handle rotations manually

    # Add items in the given order
    for idx in order:
        w0, h0 = items[idx]
        if rot_flags[idx] == 1:
            w, h = h0, w0
        else:
            w, h = w0, h0
        packer.add_rect(w, h, idx)

    # Add bins
    for _, bw, bh in panel_types:
        packer.add_bin(bw, bh, count=NB_BINS_MAX)

    packer.pack()

    bins = []
    for bin_index, b in enumerate(packer):
        bw, bh = b.width, b.height
        rects_info = []
        for rect in b:
            x, y = rect.x, rect.y
            w, h = rect.width, rect.height
            item_id = rect.rid
            rects_info.append((x, y, w, h, item_id))
        bins.append((bin_index, bw, bh, rects_info))

    return bins


def fitness_for_batch(df_batch: pd.DataFrame, order: List[int], rot_flags: List[int]) -> float:
    """
    Evaluate one individual (order + rotation flags) for a given batch.
    Higher is better.
    """
    items = list(
        zip(
            df_batch[COL_ITEM_W].astype(float).tolist(),
            df_batch[COL_ITEM_H].astype(float).tolist(),
        )
    )
    panel_types = build_panel_types_from_batch(df_batch)
    bins = pack_given_order(items, panel_types, order, rot_flags)

    # Compute utilization as fitness
    metrics = compute_waste_metrics(df_batch, bins)
    return metrics["utilization"]


# Genetic operators

def init_population(n_items: int) -> List[Tuple[List[int], List[int]]]:
    """Create an initial GA population (random permutations + random rotation flags)."""
    population = []
    for _ in range(GA_POP_SIZE):
        order = list(range(n_items))
        random.shuffle(order)
        rot_flags = [random.randint(0, 1) for _ in range(n_items)]
        population.append((order, rot_flags))
    return population


def tournament_selection(population, fitnesses):
    """Select one individual by tournament."""
    indices = random.sample(range(len(population)), GA_TOURNAMENT_SIZE)
    best_idx = max(indices, key=lambda i: fitnesses[i])
    return population[best_idx]


def crossover(parent1, parent2):
    """
    Order crossover for the permutation + single-point crossover for rotation flags.
    """
    order1, rot1 = parent1
    order2, rot2 = parent2
    n = len(order1)

    # Order crossover (OX-style)
    cut1, cut2 = sorted(random.sample(range(n), 2))
    child_order = [None] * n
    child_order[cut1:cut2] = order1[cut1:cut2]

    fill_positions = [i for i in range(n) if not (cut1 <= i < cut2)]
    fill_values = [v for v in order2 if v not in child_order]
    for pos, val in zip(fill_positions, fill_values):
        child_order[pos] = val

    # Rotation flags crossover
    cx_point = random.randint(1, n - 1)
    child_rot = rot1[:cx_point] + rot2[cx_point:]

    return child_order, child_rot


def mutate(order, rot_flags):
    """Mutate permutation (swap) and rotation flags (flip)."""
    n = len(order)

    # Small chance of swapping two positions
    if random.random() < GA_MUTATION_RATE:
        i, j = random.sample(range(n), 2)
        order[i], order[j] = order[j], order[i]

    # Small chance for each item to flip its rotation
    for i in range(n):
        if random.random() < GA_MUTATION_RATE:
            rot_flags[i] = 1 - rot_flags[i]


def evolve_population(df_batch: pd.DataFrame):
    """Run the GA loop for a single batch."""
    n_items = len(df_batch)
    population = init_population(n_items)

    # Evaluate initial population
    fitnesses = [fitness_for_batch(df_batch, order, rot) for order, rot in population]

    elite_size = max(1, int(GA_ELITE_FRACTION * GA_POP_SIZE))

    best_individual = None
    best_fitness = -1.0

    for gen in range(GA_N_GENERATIONS):
        # Elitism
        ranked_indices = sorted(range(GA_POP_SIZE), key=lambda i: fitnesses[i], reverse=True)
        new_population = [population[i] for i in ranked_indices[:elite_size]]

        # Update global best
        if fitnesses[ranked_indices[0]] > best_fitness:
            best_fitness = fitnesses[ranked_indices[0]]
            best_individual = population[ranked_indices[0]]

        # Fill the rest of the population
        while len(new_population) < GA_POP_SIZE:
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            child_order, child_rot = crossover(parent1, parent2)
            mutate(child_order, child_rot)
            new_population.append((child_order, child_rot))

        population = new_population
        fitnesses = [fitness_for_batch(df_batch, order, rot) for order, rot in population]

        print(f"Generation {gen+1}/{GA_N_GENERATIONS} — best fitness: {best_fitness:.3%}")

    return best_individual, best_fitness


def run_genetic_algorithm():
    """Run the GA-based optimizer on all batches."""
    batches = load_items_by_batch(CSV_PATH)
    for batch_id, df_batch in batches.items():
        print(f"\n=== Genetic algorithm: batch {batch_id} ===")
        best_individual, best_fit = evolve_population(df_batch)
        print(f"Best utilization found: {best_fit:.3%}")

        # Rebuild final packing to visualize
        order, rot_flags = best_individual
        items = list(
            zip(
                df_batch[COL_ITEM_W].astype(float).tolist(),
                df_batch[COL_ITEM_H].astype(float).tolist(),
            )
        )
        panel_types = build_panel_types_from_batch(df_batch)
        bins = pack_given_order(items, panel_types, order, rot_flags)
        plot_packing(bins, df_batch, title=f"GA packing — {batch_id}")


if __name__ == "__main__":
    # You can comment/uncomment depending on what you want to run
    run_baseline()
    run_genetic_algorithm()